🔥 КИЛЛЕР-ПАЙПЛАЙН: Решение любой ML-задачи за 5 минут

Этот ноутбук — твоя главная шпора на олимпиаде. Здесь минимум слов, максимум кода.

**Главная идея:** CatBoost из коробки работает ЛУЧШЕ, чем с рандомными параметрами!

---
ЧАСТЬ 1: Копипаст-решение (буквально 5 строк)

Это работает на 80% задач! Просто меняй названия файлов и таргета.

In [ ]:
# === УНИВЕРСАЛЬНЫЙ ПАЙПЛАЙН ДЛЯ КЛАССИФИКАЦИИ ===
import pandas as pd
from catboost import CatBoostClassifier

# 1. Загрузка
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 2. Определяем признаки и таргет
TARGET = 'target'  # <-- ПОМЕНЯЙ НА СВОЙ ТАРГЕТ
ID_COL = 'id'      # <-- ПОМЕНЯЙ ЕСЛИ НУЖНО

features = [c for c in train.columns if c not in [TARGET, ID_COL]]

# 3. Находим категориальные признаки автоматически
cat_features = train[features].select_dtypes(include=['object']).columns.tolist()

# 4. Обучаем модель (CatBoost сам справляется с пропусками и категориями!)
model = CatBoostClassifier(verbose=100, random_seed=42, cat_features=cat_features)
model.fit(train[features], train[TARGET])

# 5. Предсказываем и сохраняем
predictions = model.predict(test[features])
pd.DataFrame({ID_COL: test[ID_COL], TARGET: predictions}).to_csv('submission.csv', index=False)
print('Готово! ✅')

In [ ]:
# === УНИВЕРСАЛЬНЫЙ ПАЙПЛАЙН ДЛЯ РЕГРЕССИИ ===
import pandas as pd
from catboost import CatBoostRegressor

# 1. Загрузка
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 2. Определяем признаки и таргет
TARGET = 'target'  # <-- ПОМЕНЯЙ НА СВОЙ ТАРГЕТ
ID_COL = 'id'      # <-- ПОМЕНЯЙ ЕСЛИ НУЖНО

features = [c for c in train.columns if c not in [TARGET, ID_COL]]

# 3. Находим категориальные признаки автоматически
cat_features = train[features].select_dtypes(include=['object']).columns.tolist()

# 4. Обучаем модель
model = CatBoostRegressor(verbose=100, random_seed=42, cat_features=cat_features)
model.fit(train[features], train[TARGET])

# 5. Предсказываем и сохраняем
predictions = model.predict(test[features])
pd.DataFrame({ID_COL: test[ID_COL], TARGET: predictions}).to_csv('submission.csv', index=False)
print('Готово! ✅')

---
ЧАСТЬ 2: Пайплайн с валидацией

In [ ]:
# === ПАЙПЛАЙН С ВАЛИДАЦИЕЙ ===
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# Загрузка
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

TARGET = 'target'
ID_COL = 'id'
features = [c for c in train.columns if c not in [TARGET, ID_COL]]
cat_features = train[features].select_dtypes(include=['object']).columns.tolist()

# Разбиение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    train[features], train[TARGET], 
    test_size=0.2, random_state=42, stratify=train[TARGET]
)

# Обучение с early stopping
model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=6,
    verbose=200,
    random_seed=42,
    cat_features=cat_features,
    early_stopping_rounds=100
)

model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

# Оценка
y_pred = model.predict(X_val)
y_proba = model.predict_proba(X_val)[:, 1]

print(f'Accuracy: {accuracy_score(y_val, y_pred):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba):.4f}')
print(f'F1: {f1_score(y_val, y_pred):.4f}')

# Submission
predictions = model.predict(test[features])
pd.DataFrame({ID_COL: test[ID_COL], TARGET: predictions}).to_csv('submission.csv', index=False)

---
ЧАСТЬ 3: Если метрика MAE или RMSE

---
ЧАСТЬ 4: Feature Engineering — главный способ улучшить модель

FE часто важнее, чем подбор параметров. Хорошие признаки = хорошая модель.

**Типы признаков, которые почти всегда работают:**

| Тип | Когда использовать | Пример |
|-----|-------------------|--------|
| Агрегаты по группам | Есть категории | Средняя цена по категории товара |
| Отношения/разности | Есть связанные числа | price / area, end_date - start_date |
| Из дат | Есть даты | Год, месяц, день недели, час |
| Счётчики | Есть категории | Сколько раз встречается значение |
| Бинарные флаги | Есть пороговые значения | is_weekend, is_expensive |

In [ ]:
# === 1. АГРЕГАТЫ ПО ГРУППАМ ===
# Идея: для каждой категории посчитать статистику по числовому признаку
# Это помогает модели понять "насколько этот объект типичен для своей группы"

# Среднее значение числового признака по группе
train['price_mean_by_category'] = train.groupby('category')['price'].transform('mean')
test['price_mean_by_category'] = test.groupby('category')['price'].transform('mean')

# Можно добавить и другие статистики
train['price_std_by_category'] = train.groupby('category')['price'].transform('std')
train['price_max_by_category'] = train.groupby('category')['price'].transform('max')
train['price_min_by_category'] = train.groupby('category')['price'].transform('min')

# Отклонение от среднего по группе (насколько объект отличается от типичного)
train['price_diff_from_group_mean'] = train['price'] - train['price_mean_by_category']

In [ ]:
# === 2. ОТНОШЕНИЯ И РАЗНОСТИ ===
# Идея: комбинировать признаки математически
# Модель может сама это найти, но явные признаки помогают

# Отношения (деление)
train['price_per_sqm'] = train['price'] / (train['area'] + 1)  # +1 чтобы избежать деления на 0
train['efficiency'] = train['output'] / (train['input'] + 1)

# Разности
train['profit'] = train['revenue'] - train['cost']
train['age'] = 2024 - train['birth_year']

# Произведения (реже, но иногда полезно)
train['total_value'] = train['quantity'] * train['unit_price']

In [ ]:
# === 3. ПРИЗНАКИ ИЗ ДАТ ===
# Даты — это кладезь информации!

# Сначала конвертируем в datetime
train['date'] = pd.to_datetime(train['date'])

# Базовые компоненты
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['dayofweek'] = train['date'].dt.dayofweek  # 0=пн, 6=вс
train['hour'] = train['date'].dt.hour  # если есть время

# Полезные флаги
train['is_weekend'] = (train['dayofweek'] >= 5).astype(int)
train['is_month_start'] = train['date'].dt.is_month_start.astype(int)
train['is_month_end'] = train['date'].dt.is_month_end.astype(int)

# Циклическое кодирование (для месяца, дня недели, часа)
# Помогает модели понять, что декабрь (12) близок к январю (1)
import numpy as np
train['month_sin'] = np.sin(2 * np.pi * train['month'] / 12)
train['month_cos'] = np.cos(2 * np.pi * train['month'] / 12)

# День года (для сезонности)
train['dayofyear'] = train['date'].dt.dayofyear

In [ ]:
# === 4. СЧЁТЧИКИ И ЧАСТОТЫ ===
# Идея: как часто встречается значение категории?
# Редкие категории ведут себя иначе, чем частые

# Счётчик (сколько раз встречается)
category_counts = train['category'].value_counts()
train['category_count'] = train['category'].map(category_counts)
test['category_count'] = test['category'].map(category_counts)

# Флаг редкой категории
train['is_rare_category'] = (train['category_count'] < 10).astype(int)

# Частота (доля от общего числа)
train['category_freq'] = train['category_count'] / len(train)

In [ ]:
# === 5. БИНАРНЫЕ ФЛАГИ ===
# Идея: превратить числовой признак в да/нет
# Помогает когда важен факт, а не точное значение

train['has_discount'] = (train['discount'] > 0).astype(int)
train['is_expensive'] = (train['price'] > train['price'].median()).astype(int)
train['is_null_value'] = train['some_column'].isnull().astype(int)

# Пороги из предметной области
train['is_adult'] = (train['age'] >= 18).astype(int)
train['is_high_income'] = (train['income'] > 100000).astype(int)

In [ ]:
# === 6. КОМБИНАЦИИ КАТЕГОРИЙ ===
# Идея: создать новую категорию из двух существующих
# Полезно когда комбинация важнее отдельных признаков

train['city_category'] = train['city'] + '_' + train['category']
train['user_product'] = train['user_id'].astype(str) + '_' + train['product_id'].astype(str)

In [ ]:
# === УНИВЕРСАЛЬНАЯ ФУНКЦИЯ FE ===
# Копипасти и адаптируй под свою задачу

def feature_engineering(df, is_train=True):
    """Универсальный feature engineering"""
    df = df.copy()
    
    # --- Числовые признаки ---
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    # Убираем id и target если есть
    num_cols = [c for c in num_cols if c not in ['id', 'target']]
    
    if len(num_cols) >= 2:
        # Агрегаты по строке
        df['row_sum'] = df[num_cols].sum(axis=1)
        df['row_mean'] = df[num_cols].mean(axis=1)
        df['row_std'] = df[num_cols].std(axis=1)
        df['row_max'] = df[num_cols].max(axis=1)
        df['row_min'] = df[num_cols].min(axis=1)
        df['row_range'] = df['row_max'] - df['row_min']
    
    # --- Категориальные признаки ---
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    for col in cat_cols:
        # Счётчики
        counts = df[col].value_counts()
        df[f'{col}_count'] = df[col].map(counts)
    
    # --- Даты ---
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            try:
                df[col] = pd.to_datetime(df[col])
                df[f'{col}_year'] = df[col].dt.year
                df[f'{col}_month'] = df[col].dt.month
                df[f'{col}_day'] = df[col].dt.day
                df[f'{col}_dayofweek'] = df[col].dt.dayofweek
                df[f'{col}_is_weekend'] = (df[f'{col}_dayofweek'] >= 5).astype(int)
            except:
                pass
    
    return df

# Применяем
train = feature_engineering(train, is_train=True)
test = feature_engineering(test, is_train=False)

**Важно при FE:**

1. **Train и test обрабатывай одинаково!** Иначе модель сломается
2. **Счётчики и агрегаты считай по train**, потом применяй к test через `.map()`
3. **Не делай data leakage** — не используй информацию из будущего или из таргета
4. **Проверяй на валидации** — не все признаки улучшают модель

In [ ]:
# === БЫСТРЫЙ FEATURE ENGINEERING ===

def quick_fe(df):
    df = df.copy()
    
    # Числовые признаки
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    if len(num_cols) >= 2:
        df['sum_nums'] = df[num_cols].sum(axis=1)
        df['mean_nums'] = df[num_cols].mean(axis=1)
        df['std_nums'] = df[num_cols].std(axis=1)
    
    # Для дат
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            try:
                df[col] = pd.to_datetime(df[col])
                df[f'{col}_year'] = df[col].dt.year
                df[f'{col}_month'] = df[col].dt.month
                df[f'{col}_day'] = df[col].dt.day
                df[f'{col}_dayofweek'] = df[col].dt.dayofweek
            except:
                pass
    
    return df

train = quick_fe(train)
test = quick_fe(test)

---
ЧАСТЬ 5: Шпаргалка по параметрам

**Типичные ошибки:**

| Проблема | Решение |
|----------|----------|
| Переобучение | Уменьши depth, добавь l2_leaf_reg |
| Недообучение | Увеличь iterations, уменьши learning_rate |
| Долго учится | Уменьши iterations, увеличь learning_rate |

---
ЧЕКЛИСТ

✅ Загрузил данные  
✅ Определил TARGET и ID  
✅ Нашёл категориальные признаки  
✅ Обучил CatBoost  
✅ Проверил на валидации  
✅ Сделал submission  